In [ ]:
import pandas as pd
import numpy as np
import ast, json
import os
import statsmodels.api as sm
import warnings

from imblearn.pipeline import Pipeline
from numpy import where
from statistics import *
from scipy.stats import chi2_contingency
from numpy import loadtxt
from matplotlib import pyplot
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')
directory = '/content/drive/MyDrive/Research/Aqualab/Pipeline'  ###### 【Customizable item】
os.chdir(directory)

Mounted at /content/drive


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.float_format = '{:.10f}'.format
pd.set_option('display.float_format', '{:.10f}'.format)


# I. Pre-process log data

- Please skip this session if all .tsv files have already converted to .csv format for regression/chi-square test
- Please make sure that all the .tsv files are in the current working directory

In [ ]:
def data_process(data):
    # Filter out rows where app_branch is 'develop'
    data = data[data['app_branch'] != 'develop']
    data = data.reset_index(drop=True)

    # Sort values by user_id and timestamp
    data = data.sort_values(by=['user_id', 'timestamp'])

    # Filter out rows where event_name is 'load_error'
    data = data[data['event_name'] != 'load_error']
    return data

def extract_job_name(game_state):
    # extract job_name from 'game_state'
    try:
        game_state_dict = json.loads(game_state)
        return game_state_dict.get('job_name', '')
    except json.JSONDecodeError:
        return ''

def data_clean(data):
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data[data['job_name'] != 'no-active-job']
    data = data.reset_index(drop=False)
    data = data.rename(columns={'level_0': 'id_org'})
    return data

def data_chunk(data):
    # Task end when: {job differs}
    data["task_status"] = data["job_name"].shift(1,fill_value=data["job_name"].head(1)) != data["job_name"]

    # Task end when: {student changes}
    data["student_change"] = data["user_id"].shift(1, fill_value=data["user_id"].head(1)) != data["user_id"]
    data.loc[data["student_change"] == True, "task_status"] = True


    # Create task_id: whenever task status changes, task id is incremented by 1 (i.e., sum all True)
    data['task_id'] = data['task_status'].cumsum()
    data['task_id'] += 1
    data['task_id'] = data['task_id'].astype(int)

    # Drop task_ids that has only one row in the log file
    task_id_counts = data['task_id'].value_counts()
    data = data[data['task_id'].map(task_id_counts) > 1]
    if 'id_org' in data.columns:
        data = data.drop(columns=['id_org'], axis = 1)
    data = data.reset_index(drop=True)

    data = data.rename(columns={'job_name': 'job_string'})

    # Filter rows where 'event_name' is not equal to 'complete_job' for each 'task_id'
    filtered_data = data[data['event_name'] != 'complete_job']

    # Find 'task_id' where 'user_id' is the same but 'job_string' is different in the next 'task_id'
    task_ids = []
    for index, row in filtered_data.iterrows():
        current_task_id = row['task_id']
        current_user_id = row['user_id']
        current_job_string = row['job_string']

        next_row = data.iloc[index + 1] if index + 1 < len(data) else None

        if next_row is not None and next_row['user_id'] == current_user_id and next_row['job_string'] != current_job_string:
            task_ids.append(current_task_id)

    return data


def refine_data(data):
    # Identifying users to remove: those who only have 'kelp-welcome' as their job_string
    users_to_remove = data.groupby('user_id')['job_string'].apply(lambda x: all(x == 'kelp-welcome'))
    users_to_remove = users_to_remove[users_to_remove].index
    data = data[~data['user_id'].isin(users_to_remove)]

    # Remove user_id == 'default'
    data = data[data['user_id'] != 'default']

    # Adding 'task' column: counts 'complete_task' events per user per task_id
    data['task'] = data.groupby(['user_id', 'task_id'])['event_name'].transform(lambda x: x.eq('complete_task').sum())

    # Adding 'job' column: counts 'complete_job' events per user per task_id
    data['job'] = data.groupby(['user_id', 'task_id'])['event_name'].transform(lambda x: x.eq('complete_job').sum())

    # Adding 'session' column: counts 'session_id' events per user per job_string
    data['session'] = data.groupby(['user_id', 'job_string'])['session_id'].transform(lambda x: x.nunique())

    return data


def aggregate_rows(group):
    # If any event is complete_job in this task_id, then this task_id has event_change column as 'Not Swapped'
    if any(event == 'complete_job' for event in group['event_name'].values):
        group['event_change'] = 'Not Swapped'
    # If no event is complete_job in this task_id, then this task_id has event_change column as 'Swapped'
    else:
        group['event_change'] = 'Swapped'

    total_time_diff = 0

    # Loop through each session_id within the group
    for session_id, session_group in group.groupby('session_id'):
        # Calculate the time difference for the current session_id
        time_diff = session_group['timestamp'].max() - session_group['timestamp'].min()
        total_time_diff += abs(time_diff.total_seconds())  # Add the absolute value of time difference in seconds to total

    # Assign the total time difference to the group
    group['time_diff'] = total_time_diff

    # Return the first row of the group with the required columns including the calculated total time difference
    return group[['user_id', 'job_string', 'task_id', 'event_change', 'time_diff', 'session', 'task', 'job']].iloc[0]


# Function to update the last row of each user_id
def update_last_row(group):
    group.iloc[-1, group.columns.get_loc('event_change')] = 'Not Swapped'
    return group



# Takes all .tsv file in the working directory and convert to .csv
for filename in os.listdir('.'):

    if filename.endswith('.tsv'):

        data = pd.read_csv(filename, sep='\t')

        data = data_process(data)
        data['job_name'] = data['game_state'].apply(extract_job_name)
        data = data_clean(data)
        data = data_chunk(data)


        data = refine_data(data)

        data = data.groupby('task_id').apply(aggregate_rows).reset_index(drop=True)
        data = data.groupby('user_id').apply(update_last_row)
        data = data.reset_index(drop=True)

        data.to_csv(f"progression_{filename.split('.')[0]}.csv", index=False)
        files.download(f"progression_{filename.split('.')[0]}.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# II. Identify Target Jobs

In [ ]:
def concatenate_csv_files():
    # Get all .csv files in the current working directory
    csv_files = [file for file in os.listdir('.') if file.endswith('.csv')]

    dfs = []

    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dfs.append(df)
        concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

data = concatenate_csv_files()
not_completed_counts = data[data['event_change'] != 'Not Swapped'].groupby('job_string').size().reset_index(name='not_completed_count')
total_counts = data.groupby('job_string').size().reset_index(name='total_count')
percentage_not_completed = pd.merge(not_completed_counts, total_counts, on='job_string')
percentage_not_completed['percentage_not_completed'] = (percentage_not_completed['not_completed_count'] / percentage_not_completed['total_count']) * 100
top_percentage = percentage_not_completed.nlargest(100, 'percentage_not_completed')
top_percentage = top_percentage[top_percentage['not_completed_count'] >= 30]

# III. Chi-square test

- Please make sure that the .txt file containing all job difficulty parameters is in the current working directory.
- Level B (LevelB) is customizable.
- Criteria for selecting Level As is customizable (threshold_yes, threshold_no).
- At the end of the session, the program sorts all previous levels based on chi-square statistics from high to low. It retains only a customizable number, n, of previous jobs for Session III.

In [ ]:
def diff_parameter(path):
    para_list = []

    with open(path, 'r') as file:
        data = file.read()

    # Split the data into jobs
    jobs = data.split('\n')

    for job in jobs:
        if job.startswith('Job:'):
            job_info = {'Job': '', 'Experimentation': 0, 'Modeling': 0, 'Argumentation': 0}
            job_info['Job'] = job.split('Job:')[1].strip()
        elif job.startswith('\tExperimentation:'):
            job_info['Experimentation'] = int(job.split('\tExperimentation:')[1].strip())
        elif job.startswith('\tModeling:'):
            job_info['Modeling'] = int(job.split('\tModeling:')[1].strip())
        elif job.startswith('\tArgumentation:'):
            job_info['Argumentation'] = int(job.split('\tArgumentation:')[1].strip())

        para_list.append(job_info)  # Append the dictionary to the list

    # Convert the list of dictionaries into a DataFrame
    para = pd.DataFrame(para_list)

    return para



def process_progression_data(data):

    data['time_diff'] = pd.to_numeric(data['time_diff'], errors='coerce')
    data['job'] = pd.to_numeric(data['job'], errors='coerce')
    data['task'] = pd.to_numeric(data['task'], errors='coerce')

    data = data.drop(["task_id"], axis=1)
    data = data.drop_duplicates()

    data.reset_index(drop=True, inplace=True)

    # Replace "Not Swapped" with NaN, and "Swapped" with "switched"
    data = data.replace("Not Swapped", np.nan)
    data = data.replace("Swapped", "switched")

    # Combine 'job_string' and 'event_change' into 'job_status'
    data['job_status'] = data.apply(lambda row: str(row['job_string']) + f" ({str(row['event_change'])})" if not pd.isna(row['event_change']) else row['job_string'], axis=1)

    # Replace NaN values with "completed"
    data = data.replace(np.nan, "completed")

    # Filter out rows with 'job_string' equal to 'kelp-welcome' and reset index
    data = data[data['job_string'] != 'kelp-welcome'].reset_index(drop=True)

    # Filter out users with less than 2 entries and reset index
    data = data.groupby('user_id').filter(lambda x: len(x) >= 2).reset_index(drop=True)

    # Function to add prefix to some 'job_string'
    def add_prefix(job_string):
        if job_string in ['displaced-reef', 'turtle-danger', 'turtle-danger2']:
            return 'bayou-' + job_string
        elif job_string in ['final-final', 'above-n-below', 'completed']:
            return 'arctic-' + job_string
        else:
            return job_string

    data['job_string'] = data['job_string'].apply(add_prefix)

    return data

# Create a dictionary to count for the number of player who completed and did not complete each job
def create_job_dict(data, levelB):
    job_dict = {}
    for job in data['job_string'].unique():
        job_dict[job] = {'yes': 0, 'no': 0}


    for user, group in data.groupby('user_id'):
        # Find the minimum index for 'LevelB' job
        levelb_index = group[group['job_string'] == LevelB].index.min()

        if pd.notna(levelb_index):
          # Check each job in job_dict
          for job in job_dict:
              # Find rows before levelb_index for this user with the current job
              job_rows = group[(group.index < levelb_index) & (group['job_string'] == job)]

              # Check if any row has 'completed' event_change
              if not job_rows.empty and (job_rows['event_change'] == 'completed').any():
                  job_dict[job]['yes'] += 1
              else:
                  job_dict[job]['no'] += 1

    return job_dict

def add_values_from_para(data, para):
    argumentation_values = []
    modeling_values = []
    experimentation_values = []

    for index, row in data.iterrows():
        job_string = row['job_string']

        # Find the row in 'para' where 'Job' matches 'job_string'
        matching_row = para[para['Job'] == job_string]

        argumentation_values.append(matching_row['Argumentation'].iloc[0] if not matching_row.empty else None)
        modeling_values.append(matching_row['Modeling'].iloc[0] if not matching_row.empty else None)
        experimentation_values.append(matching_row['Experimentation'].iloc[0] if not matching_row.empty else None)

    data['Argumentation'] = argumentation_values
    data['Modeling'] = modeling_values
    data['Experimentation'] = experimentation_values

    return data

def get_level_A_packs(job_dict, threshold_yes, threshold_no):
    # Filter job A base on the threshold
    LevelApacks = []
    for job in job_dict:
        if job_dict[job]['yes'] >= threshold_yes and job_dict[job]['no'] >= threshold_no:

            LevelApacks.append(job)
    return LevelApacks


def process_macro_counts(data, LevelApacks, LevelB):
    macro_user_list_yes = []
    macro_user_list_no = []
    macro_counter_yes_switch = []
    macro_counter_no_switch = []

    for LevelA in LevelApacks:
        count_yes = 0
        user_list_yes = []

        count_no = 0
        user_list_no = []

        for user, group in data.groupby('user_id'):
            levelb_index = group[group['job_string'] == LevelB].index.min()

            if pd.notna(levelb_index):
                if (group['job_string'] == LevelA).any() and (group.index < levelb_index).any():
                    count_yes += 1
                    user_list_yes.append(user)

                if LevelA not in group.loc[group.index < levelb_index, 'job_string'].values:
                    count_no += 1
                    user_list_no.append(user)

        macro_user_list_yes.append(count_yes)
        macro_user_list_no.append(count_no)

        counter_yes_switch = 0

        for user_id in user_list_yes:
            user_group = data[(data['user_id'] == user_id) & (data['job_string'] == LevelB)]
            if 'completed' not in user_group['event_change'].values:
                counter_yes_switch += 1

        macro_counter_yes_switch.append(counter_yes_switch)

        counter_no_switch = 0

        for user_id in user_list_no:
            user_group = data[(data['user_id'] == user_id) & (data['job_string'] == LevelB)]
            if 'completed' not in user_group['event_change'].values:
                counter_no_switch += 1

        macro_counter_no_switch.append(counter_no_switch)

    return macro_user_list_yes, macro_user_list_no, macro_counter_yes_switch, macro_counter_no_switch


In [ ]:
def process_dataframe(LevelApacks, LevelB, macro_user_list_yes, macro_user_list_no, macro_counter_yes_switch, macro_counter_no_switch, n):

    df = pd.DataFrame(columns=['Level A', 'Level B', 'Played A before B', 'Count', '# Switched B'])



    # Populate the DataFrame
    for i, level_a in enumerate(LevelApacks):
        for j in range(2):
            level_b = LevelB
            played_before_b = 'Y' if j == 0 else 'N'
            count = macro_user_list_yes[i] if j == 0 else macro_user_list_no[i]
            switch_b = macro_counter_yes_switch[i] if j == 0 else macro_counter_no_switch[i]

            print(level_a, macro_counter_yes_switch[i], macro_counter_no_switch[i])

            new_row = pd.DataFrame({'Level A': [level_a], 'Level B': [level_b], 'Played A before B': [played_before_b], 'Count': [count], '# Switched B': [switch_b]})
            df = pd.concat([df, new_row], ignore_index=True)

    if (df['# Switched B'] == 0).all():
        return pd.DataFrame(), []

    df.reset_index(drop=True, inplace=True)

    df['% Switched B'] = df['# Switched B'] / df['Count']

    df['Difference'] = df.groupby('Level A')['% Switched B'].diff()
    df['Difference'] = df.groupby('Level A')['Difference'].transform(lambda x: x.fillna(x.mean()))

    df = df.sort_values(by=['Difference', 'Level A', 'Played A before B'], ascending=[False, True, False])

    df['% Switched B'] = df['% Switched B'] * 100
    df['% Switched B'] = df['% Switched B'].map('{:.2f}%'.format)

    df['Difference'] = df['Difference'] * 100
    df['Difference'] = df['Difference'].map('{:.2f}%'.format)
    df.insert(5, "# Not Switched B", df['Count'] - df['# Switched B'])

    df_grouped = df.groupby("Level A")[['# Switched B', '# Not Switched B']].transform('sum')
    df['Expected'] = ((df['# Switched B'] + df['# Not Switched B']) * df_grouped['# Switched B']) / (df_grouped['# Switched B'] + df_grouped['# Not Switched B'])

    unique_levels = df['Level A'].unique()
    chi_stats = []
    p_values = []

    for level in unique_levels:
        subset = df[df['Level A'] == level]
        observed = subset['# Switched B'].tolist()
        expected = subset['Expected'].tolist()

        observed_forchi2 = subset[['# Switched B', '# Not Switched B']].copy()

        # Perform chi-square test
        chi2, p, _, _ = chi2_contingency(observed_forchi2)
        chi_stats.append(chi2)
        p_values.append(p)

    result_df = pd.DataFrame({'Level A': unique_levels, 'Chi-Square Statistic': chi_stats, 'p-value': p_values})
    df = df.merge(result_df, on='Level A', how='left')

    ######### Modify the DataFrame to keep only the first n*2 rows #########
    ######### Extract and return the first n "Level A" values #########


    if len(df) < 2 * n:
      # If it does, keep whatever rows it has
        df = df
        level_a_first_n = df['Level A'].unique().tolist()
    else:
      # If it has 2*n rows or more, keep only the first n*2 rows
        df = df.iloc[:n*2]
        level_a_first_n = df['Level A'].iloc[range(0, n*2, 2)].unique().tolist()

    return df, level_a_first_n





In [ ]:
path_to_file = 'JobText.txt'
para = diff_parameter(path_to_file)
data = process_progression_data(data)
data = add_values_from_para(data, para)
final_dataframe = pd.DataFrame()
level_a_first_n_dict = {}


for job in top_percentage['job_string']:

    LevelB = job
    job_dict = create_job_dict(data, LevelB)
    LevelApacks = get_level_A_packs(job_dict, threshold_yes=30, threshold_no=30)
    macro_user_list_yes, macro_user_list_no, macro_counter_yes_switch, macro_counter_no_switch = process_macro_counts(data, LevelApacks, LevelB)
    result_dataframe, level_a_first_n = process_dataframe(LevelApacks, LevelB, macro_user_list_yes, macro_user_list_no, macro_counter_yes_switch, macro_counter_no_switch, n=3)
    final_dataframe = pd.concat([final_dataframe, result_dataframe])
    level_a_first_n_dict[job] = level_a_first_n


final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe


# IV. Construct regression table

- This section only consider the n number (defined in Session II) of previous jobs prior to the target job with the highest chi-square coefficient to compile the regression table.

In [ ]:
def process_levels(data, LevelApacks, LevelB, num_check):
    num_levelA = 1
    final_dataframe = pd.DataFrame()


    for LevelA in LevelApacks:
        if num_levelA > num_check:
            break

        if LevelA == "kelp-shop-welcome":
            continue

        print("**--------------------------------", LevelA, "--------------------------------**")

        x = pd.DataFrame(columns=['user_id', 'Level', 'Complete_A', 'Complete_B',
                                  'Arg_above_2', 'Mod_above_2', 'Exp_above_2',
                                  'Time', 'Job', 'Task', 'Biome', 'Session'])

        # Filter user IDs for LevelB
        user_ids_levelb = data[data['job_string'] == LevelB]['user_id'].unique()

        # Iterate over each user_id
        for user_id in user_ids_levelb:
            user_data = data[data['user_id'] == user_id]
            levelb_index = user_data[user_data['job_string'] == LevelB].index.min()


            #【Feature 1-2: Complete_A, Complete_B】: Calculate Complete_A and Complete_B

            # Identify rows where both conditions are met
            complete_a = 1 if any((user_data['job_string'] == LevelA) & (user_data['event_change'] == 'completed')) else 0
            complete_b = 1 if any((user_data['job_string'] == LevelB) & (user_data['event_change'] == 'completed')) else 0


            #【Feature 3-5: unique_job_arg_count, unique_job_mod_count, unique_job_exp_count】: Count the number of jobs [completed] where each difficulty parameter > 2
            # Select rows up to and including the first 'LevelB' occurrence
            levelb_index = user_data[user_data['job_string'] == LevelB].index.min()
            user_data_levelb = user_data.loc[:levelb_index]

            filtered_df = user_data_levelb[user_data_levelb['event_change'] == 'completed']   # Which jobs has the student completed so far?
                                                                                              # Please change 'filtered_df' to 'user_data_levelb' if we want to include any job student has accepted

            unique_job_arg_count = filtered_df[filtered_df['Argumentation'] > 2]['job_string'].nunique()
            unique_job_mod_count = filtered_df[filtered_df['Modeling'] > 2]['job_string'].nunique()
            unique_job_exp_count = filtered_df[filtered_df['Experimentation'] > 2]['job_string'].nunique()

            #【Feature 6: time_diff】: Time spent so far in hours
            time_diff  = user_data['time_diff'].sum() / 3600

            #【Feature 7-8: job_sum, task_sum】: Sum Job and Task student has completed
            job_sum = user_data_levelb['job'].sum()
            task_sum = user_data_levelb['task'].sum()

            #【Feature 9: Biome】: Number of Biome Student has been to
            biome = user_data_levelb['job_string'].str.split('-').str[0].nunique()

            #【Feature 10: time_diff】: Time paused (unique session id) so far
            session  = user_data['session'].sum() - 1


            # Append to DataFrame x
            new_row = {'user_id': user_id, 'Level': LevelA, 'Complete_A': complete_a, 'Complete_B': complete_b,
                      'Arg_above_2': unique_job_arg_count, 'Mod_above_2': unique_job_mod_count, 'Exp_above_2': unique_job_exp_count,
                      'Time': time_diff, 'Job': job_sum, 'Task': task_sum, 'Biome': biome, 'Session': session}


            x = pd.concat([x, pd.DataFrame([new_row])], ignore_index=True)

        # Drop rows where any NaN values exist due to incomplete log file
        x = x.dropna()

        # If Biome is the same all the time, drop it
        if x['Biome'].nunique() == 1:
          x = x.drop_duplicates(subset='Biome', keep=False)

        # Selecting the independent variables (excluding 'user_id' and 'Complete_B')
        X = x.drop(['user_id', 'Complete_B', 'Level'], axis=1)

        # Adding a constant to the model (intercept)
        X = sm.add_constant(X)

        # The dependent variable
        y = x['Complete_B']


        model = sm.Logit(y.astype(float), X.astype(float))
        result = model.fit(method='bfgs', maxiter=100, tol=1e-2)


        # Print the p-value for 'Complete_A'
        # p_value_complete_a = result.pvalues['Complete_A']
        # print(f"P-value for {LevelA}: {p_value_complete_a}")

        num_levelA += 1

        # Access the summary data
        summary_data = result.summary2().tables[1]

        # Convert the summary data to a DataFrame
        df_summary = pd.DataFrame(summary_data)
        level_a_row = pd.DataFrame({'LevelA': [LevelA]})

        final_dataframe = pd.concat([final_dataframe, level_a_row, df_summary])


    return final_dataframe


final_dataframe = pd.DataFrame()

for LevelB in level_a_first_n_dict:
  level_b_row = pd.DataFrame({'LevelB': [LevelB]})
  final_dataframe = pd.concat([final_dataframe, level_b_row])
  LevelApacks = level_a_first_n_dict[LevelB]

  result_dataframe = process_levels(data, LevelApacks, LevelB, num_check = 300)  ###### 【Customizable item】
  final_dataframe = pd.concat([final_dataframe, result_dataframe])


final_dataframe = final_dataframe.reset_index()
final_dataframe.rename(columns={'index': 'Variable'}, inplace=True)
final_dataframe['Variable'] = final_dataframe['Variable'].replace(0, np.nan)
columns_ordered = ['LevelB', 'LevelA', 'Variable', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]']
final_dataframe = final_dataframe[columns_ordered]


In [ ]:
final_dataframe

,LevelB,LevelA,Variable,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,coral-hunting-lions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,bayou-reef-decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,const,-0.0557888943,0.6609289513,-0.0844098208,0.9327305993,-1.3511858352,1.2396080466
3,NaN,NaN,Complete_A,1.5536130657,0.3091909833,5.0247683460,0.0000005040,0.9476098741,2.1596162573
4,NaN,NaN,Arg_above_2,-0.0748432561,0.0969113920,-0.7722854307,0.4399453925,-0.2647860940,0.1150995819
...,...,...,...,...,...,...,...,...,...
1173,NaN,NaN,Job,1.2359287238,0.3507628849,3.5235447561,0.0004258152,0.5484461022,1.9234113453
1174,NaN,NaN,Task,-0.1614867305,0.1777164633,-0.9086762559,0.3635210412,-0.5098045980,0.1868311370
1175,NaN,NaN,Biome,-0.9317869148,0.3132586939,-2.9744965831,0.0029346972,-1.5457626727,-0.3178111568
1176,NaN,NaN,Session,-0.0024605352,0.0083645911,-0.2941608498,0.7686349951,-0.0188548324,0.0139337620
